In [97]:
import numpy as np
import pandas as pd

### Pdf reading

In [98]:
import PyPDF2

In [99]:
def _convert_pdf_to_text(path):
    """Convert the PDF to text and store it in the self.content
    attribute.
    """
    text = []
    with open(path, 'rb') as f:
        # create a PDF object
        pdf = PyPDF2.PdfReader(f)
        # iterate over every page in the PDF
        for page in range(len(pdf.pages)):
            # get the page object
            page_obj = pdf.pages[page]
            # extract text from the page
            text.append(page_obj.extract_text())
    text = "\n".join(text)
    return text

In [100]:
text = _convert_pdf_to_text("D:\Projects\LLM\Createdb\docs\A Recent trend in SHM.pdf")

In [101]:
print(text)

http://ammtiac.alionscience.com TheAMMTIACQuarterly,Volume3,Number4 3
INTRODUCTION
Thesubjectofstructuralhealthmonitoring(SHM)isemergingasanincreasinglyimportantcomponentofoverallNondestructiveEvaluation (NDE) programs and is now being considered forimplementationinavarietyofapplicationsincludingspacecraftcomponents, bridges, and aircraft. Over the past several years,
therehavebeenanumberoflimiteddemonstrationsofSHMinactualfieldapplications.InadditiontotheArmy,NavyandAirForce,otherfederalagenciesinterestedinthistopicincludethe
FederalAviationAdministration,NASAandtheDepartmentofTransportation.Thedetectionofdamageatanylocationisacom-
monchallengeinmostlargestructures,andthusSHMhaspoten-tial applications across a range of industrial sectors. Examplesincludethedetectionofimpactinduceddelaminationsincompos-
ite aerospace structures, localized corrosion in petrochemical
plantsandunauthorizedpenetrationsofshippingcontainers.
SuccessfulfieldimplementationofSHMhasnotbeenwidely
achievedanddepen

In [102]:
data = [[1,text]]

In [103]:
df = pd.DataFrame(data, columns=['id','raw_text'])

In [104]:
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

C:\Users\sai18\AppData\Local\Temp\ipykernel_6112\2340421553.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


### Punctuation removal

In [105]:
import string
import re 
# import nltk
# from nltk.corpus import stopwords


data_file_path = "data.txt"
with open(data_file_path, "r", encoding="utf-8") as file:
    data_text = file.read()
    
words = []
word = ''


print(len(data_text))
#punctuation

def pc(data_text):
    data_text = data_text.split('\"' and "," and ":")
    text = ""
    for i in data_text:
        text = text + i
    print(len(text))

    text = ""

    for i in data_text:
        text = text + i
    
    return text

df['pc_removal'] = df['raw_text'].apply(pc)


29592
21133


### url

In [142]:
# #url
url_pattern = r'https?:?//\S+'

def add_urls(text):
    httplinks = re.findall(url_pattern,text)
    return httplinks
url_pattern = re.compile(r'https?:?//\S+')
def remove_urls(text):
    return url_pattern.sub('', text)

df['urls'] = df['pc_removal'].apply(add_urls)
df['without_url'] = df['pc_removal'].apply(remove_urls)

### 

In [108]:

# for i in text[:100]:
#     if(i == ' '):
#         words. append(word)
#         word = ''
#         continue
#     else:
#         word = word + i
# print(words, "These are words")

# words_new = []

# for i in words:
#     if "\n" in i:
#         words = i.split("\n")
#         for j in words:
#             words_new.append(j)
#     else:
#         words_new.append(i)
# print(words_new, "These are new words")

# #tokenization


# #stop words

# stop_words = set(stopwords.words('english'))
# df['Message'] = df['Message'].apply(lambda x: [word for word in x if word not in stop_words])

# #lemmetization and stemming

# #numerical data

# #citations, references and footnotes

# #latex Expressions

# #sentence segmentation

# #doccument structure

## tokenizing

In [109]:
from nltk.tokenize import word_tokenize

new = word_tokenize(text)

In [110]:
import numpy as np
from wordfreq import get_frequency_dict

word_prob = get_frequency_dict(lang='en', wordlist='large')
max_word_len = max(map(len, word_prob))  # 34

def viterbi_segment(text, debug=False):
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        new_probs = []
        for j in range(max(0, i - max_word_len), i):
            substring = text[j:i]
            length_reward = np.exp(len(substring))
            freq = word_prob.get(substring, 0) * length_reward
            compounded_prob = probs[j] * freq
            new_probs.append((compounded_prob, j))
            
            if debug:
                print(f'[{j}:{i}] = "{text[lasts[j]:j]} & {substring}" = ({probs[j]:.8f} & {freq:.8f}) = {compounded_prob:.8f}')

        prob_k, k = max(new_probs)  # max of a touple is the max across the first elements, which is the max of the compounded probabilities
        probs.append(prob_k)
        lasts.append(k)

        if debug:
            print(f'i = {i}, prob_k = {prob_k:.8f}, k = {k}, ({text[k:i]})\n')


    # when text is a word that doesn't exist, the algorithm breaks it into individual letters.
    # in that case, return the original word instead
    if len(set(lasts)) == len(text):
        return text

    words = []
    k = len(text)
    while 0 < k:
        word = text[lasts[k]:k]
        words.append(word)
        k = lasts[k]
    words.reverse()
    return ' '.join(words)

def split_message(message):
  new_message = ' '.join(viterbi_segment(wordmash, debug=False) for wordmash in message.split())
  return new_message

In [111]:
df['string_slicing'] = df['raw_text'].apply(split_message)

In [112]:
text 

'http://ammtiac.alionscience.com TheAMMTIACQuarterly,Volume3,Number4 3\nINTRODUCTION\nThesubjectofstructuralhealthmonitoring(SHM)isemergingasanincreasinglyimportantcomponentofoverallNondestructiveEvaluation (NDE) programs and is now being considered forimplementationinavarietyofapplicationsincludingspacecraftcomponents, bridges, and aircraft. Over the past several years,\ntherehavebeenanumberoflimiteddemonstrationsofSHMinactualfieldapplications.InadditiontotheArmy,NavyandAirForce,otherfederalagenciesinterestedinthistopicincludethe\nFederalAviationAdministration,NASAandtheDepartmentofTransportation.Thedetectionofdamageatanylocationisacom-\nmonchallengeinmostlargestructures,andthusSHMhaspoten-tial applications across a range of industrial sectors. Examplesincludethedetectionofimpactinduceddelaminationsincompos-\nite aerospace structures, localized corrosion in petrochemical\nplantsandunauthorizedpenetrationsofshippingcontainers.\nSuccessfulfieldimplementationofSHMhasnotbeenwidely\nachiev

In [113]:
def save_text_to_file(text, file_path):
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text)

In [114]:

output_file_path = "data.txt"
save_text_to_file(df['string_slicing'][0], output_file_path)